In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import pixel as pix
import utilities as util
import tile
import resnet as rs

Using TensorFlow backend.


In [2]:
landsat, sentinel, dem, label = util.load_data()
tile_size = 64
num_classes = util.get_class_count()
pixel_gen = pix.pixel_gen(landsat, sentinel, dem, label, tile_size, num_classes)
tile_gen = tile.tile_gen(landsat, sentinel, dem, label, tile_size, num_classes)

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"]="3";

In [4]:
pixel_gen.gen_pixels(20000, index = 4, balanced=True)

100%|██████████| 1/1 [00:45<00:00, 45.73s/it]

pixels generated 13064


In [5]:
epochs = 200
input_shape = tile_gen.get_input_shape(reshape=True)
print(input_shape)


TypeError: get_input_shape() got an unexpected keyword argument 'reshape'

In [ ]:
model = rs.ResnetBuilder.build_resnet_34(input_shape,num_classes)
opt = Adam()
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit_generator(generator=tile_gen.tile_generator(train_px, batch_size), 
                    steps_per_epoch=len(train_px) // batch_size, epochs=epochs, verbose=1,
                    validation_data=tile_gen.tile_generator(val_px, batch_size),
                    validation_steps=len(val_px) // batch_size)